
# Proyecto de SpakSQL con Power BI: Análisis de Ventas de una Tienda Online

## 1. Importación de librerías


In [1]:
!pip install findspark


In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import col, isnull, when, count, regexp_replace, desc, to_date, date_format
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

## 2. Inicialización de pyspark y contexto sql

In [3]:
sc = SparkSession.builder\
        .master("local")\
        .appName("Análisis de Ventas Online")\
        .config("spark.logLevel", "WARN")\
        .getOrCreate()

In [4]:
sqlContext = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
sc

## 3. Carga de datos

In [6]:
ventas = sc.read.csv("Online Retail.csv", header=True, inferSchema=True)
ventas.registerTempTable('ventas')

/usr/local/spark/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


## 4. Exploración de los Datos

In [7]:
ventas.printSchema()


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [8]:
# Contar el número de filas y columnas
num_filas = ventas.count()
num_columnas = len(ventas.columns)

# Imprimir el número de filas y columnas
print("Loaded dataset contains %d rows" % num_filas)
print("Loaded dataset contains %d columns" % num_columnas)



Loaded dataset contains 541909 rows
Loaded dataset contains 8 columns


In [9]:
# Mostrar las primeras filas para obtener una visión general de los datos
ventas.show(5)

# Describir los datos para obtener estadísticas básicas
ventas.describe().show()

# Contar el número de filas totales y el número de filas únicas
total_rows = ventas.count()
unique_rows = ventas.distinct().count()
print(f"Total de filas: {total_rows}, Filas únicas: {unique_rows}")

# Verificar si hay valores nulos en las columnas críticas
ventas.select([count(when(isnull(c), c)).alias(c) for c in ventas.columns]).show()


+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01/12/2010 8:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01/12/2010 8:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01/12/2010 8:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01/12/2010 8:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01/12/2010 8:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows

+-------+------------------+-----------

Al observar las primeras filas, identificamos una variedad de productos comercializados, tales como decoraciones y accesorios para el hogar. La información recopilada abarca detalles desde el código y descripción del producto hasta la identificación del cliente y el país de venta, pasando por datos de facturación y precios por unidad.

Descripción Estadística: Detallamos aquí algunos puntos clave:

* InvoiceNo: Este identificador de factura es una secuencia de caracteres, sin valores faltantes.
* StockCode: Código del producto, también sin ausencias.
* Description: Se detectaron 1,454 entradas nulas que requieren atención.
* Quantity: Presenta extremos que sugieren posibles errores de entrada o devoluciones incorrectamente registradas.
* InvoiceDate: Esta fecha no presenta valores nulos.
* UnitPrice: Observamos valores extremos y negativos que podrían indicar errores.
* CustomerID: Con 135,080 valores nulos, esta ausencia compromete análisis de comportamiento del cliente.
* Country: No presenta valores nulos.
* Calidad de Datos: Un aspecto preocupante es la cantidad significativa de valores nulos en 'CustomerID' y 'Description', así como la presencia de duplicados que superan el número de filas únicas.

Pasos Sugeridos para la Limpieza y Análisis:

Tratamiento de Valores Nulos: Decidir si eliminar las filas con descripciones nulas y cómo imputar los valores de 'CustomerID'.
Corrección de Valores Anómalos: Investigar y corregir los valores extremos en 'Quantity' y 'UnitPrice', asegurando la positividad y formato correcto de los últimos.
Eliminación de Duplicados: Determinar su relevancia y proceder según convenga.
Análisis Exploratorio Sugerido:

Ventas por Producto y Descripción: Identificar los más vendidos.
Análisis Temporal: Examinar variaciones y tendencias estacionales.
Distribución Geográfica: Analizar cómo las ventas y precios varían entre países.
Al limpiar y analizar meticulosamente estos datos, se desvelarán insights valiosos que contribuirán a la toma de decisiones estratégicas y a la optimización de procesos en la empresa.

## 5. Limpieza de los Datos

Primero, nos asegúraremos de que UnitPrice esté en un formato decimal adecuado, convirtiendo comas en puntos y asegurándonos de que sea positivo:

In [10]:
# Convertir comas en puntos en el campo UnitPrice y asegurarse de que sea un valor numérico
ventas = ventas.withColumn("UnitPrice", regexp_replace(col("UnitPrice"), ",", ".").cast("double"))

# Eliminar valores negativos o cero, ya que no son válidos para un precio unitario
ventas = ventas.filter(col("UnitPrice") > 0)

Vamos a asegurarnos de que la columna InvoiceDate está correctamente convertida a un tipo de fecha

In [11]:
ventas.select("InvoiceDate").show(10, truncate=False)

+---------------+
|InvoiceDate    |
+---------------+
|01/12/2010 8:26|
|01/12/2010 8:26|
|01/12/2010 8:26|
|01/12/2010 8:26|
|01/12/2010 8:26|
|01/12/2010 8:26|
|01/12/2010 8:26|
|01/12/2010 8:28|
|01/12/2010 8:28|
|01/12/2010 8:34|
+---------------+
only showing top 10 rows



In [12]:
from pyspark.sql.functions import to_date, date_format

# Convertir la columna InvoiceDate a tipo fecha usando el formato correcto
ventas = ventas.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "dd/MM/yyyy H:mm"))

# Crear una nueva columna Month extrayendo el mes de la columna InvoiceDate
ventas = ventas.withColumn("Month", date_format(col("InvoiceDate"), "yyyy-MM"))


# Verificar la conversión de la fecha
ventas.select("Month").show(10)


+-------+
|  Month|
+-------+
|2010-12|
|2010-12|
|2010-12|
|2010-12|
|2010-12|
|2010-12|
|2010-12|
|2010-12|
|2010-12|
|2010-12|
+-------+
only showing top 10 rows



Ahora vamos a eliminar los duplicados para asegurarnos de que cada entrada sea única y relevante:

In [13]:
ventas = ventas.dropDuplicates()


Asignar un ID único para todos los valores nulos, como -1. Esto puede ayudar a realizar análisis que incluyan el conteo de transacciones, pero no afectará el análisis centrado en comportamientos específicos de clientes identificados.

In [14]:
ventas = ventas.na.fill({"CustomerID": -1})


Rellenar valores nulos en Description con "Unknown" garantiza que todas las filas se mantengan en el análisis, evita errores y proporciona consistencia y completitud en los resultados, lo que es fundamental para tomar decisiones informadas y precisas en el negoci

In [15]:
# Rellenar valores nulos en Description con un valor específico (por ejemplo, 'Unknown')
ventas = ventas.na.fill({"Description": "Unknown"})

Ahora vamos a investigar sobre los valores extremos de "Quantity" para entender su distribución y contexto:

In [16]:
# Visualizar las 10 transacciones más grandes y más pequeñas en términos de cantidad
ventas.orderBy("Quantity").show(10)
ventas.orderBy(desc("Quantity")).show(10)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|  Month|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-------+
|  C581484|    23843|PAPER CRAFT , LIT...|  -80995| 2011-12-09|     2.08|     16446|United Kingdom|2011-12|
|  C541433|    23166|MEDIUM CERAMIC TO...|  -74215| 2011-01-18|     1.04|     12346|United Kingdom|2011-01|
|  C536757|    84347|ROTATING SILVER A...|   -9360| 2010-12-02|     0.03|     15838|United Kingdom|2010-12|
|  C550456|    21108|FAIRY CAKE FLANNE...|   -3114| 2011-04-18|      2.1|     15749|United Kingdom|2011-04|
|  C550456|    21175|GIN + TONIC DIET ...|   -2000| 2011-04-18|     1.85|     15749|United Kingdom|2011-04|
|  C550456|   85123A|WHITE HANGING HEA...|   -1930| 2011-04-18|     2.55|     15749|United Kingdom|2011-04|
|  C556522|    22920|   HERB

Los datos que has proporcionado muestran claramente que los valores extremos en la columna Quantity representan tanto compras grandes como devoluciones grandes (negativos). Las entradas con cantidades muy grandes y su correspondiente devolución, como en los casos de 23843 y 23166, sugieren transacciones válidas seguidas de devoluciones completas. Esto podría ser indicativo de problemas con productos específicos, errores en el pedido, o decisiones de cambio en la gestión de inventario. Las devoluciones (cantidades negativas) y las compras grandes son parte normal de la operación comercial para muchos negocios, especialmente en el comercio al por mayor o en situaciones de pedidos en masa que son cancelados o devueltos por alguna razón.

Dada la importancia de ambos tipos de transacciones (positivas y negativas), vamos a Separar Compras y Devoluciones
Considere separar las transacciones en compras y devoluciones para análisis específicos. Esto permite analizar las tendencias de compra y devolución por separado, lo cual puede ser útil para entender el comportamiento del cliente y la calidad del producto.

In [17]:
purchases = ventas.filter(col("Quantity") >= 0)
returns = ventas.filter(col("Quantity") < 0)

# Contar filas en los DataFrames filtrados
total_filas_purchases = purchases.count()
total_filas_returns = returns.count()

print(f"Total de filas en compras: {total_filas_purchases}")
print(f"Total de filas en devoluciones: {total_filas_returns}")
print(f"Suma de filas en compras y devoluciones: {total_filas_purchases + total_filas_returns}")


Total de filas en compras: 524877
Total de filas en devoluciones: 9251
Suma de filas en compras y devoluciones: 534128


In [18]:
# Guardar los DataFrames limpios en archivos CSV con modo sobrescritura
purchases.write.mode("overwrite").csv("purchases_clean.csv", header=True)
returns.write.mode("overwrite").csv("returns_clean.csv", header=True)

In [19]:
# Crear vistas temporales para realizar consultas SQL
purchases.createOrReplaceTempView("purchases")
returns.createOrReplaceTempView("returns")
ventas.createOrReplaceTempView("ventas")
ventas.withColumn("Month", date_format(col("InvoiceDate"), "yyyy-MM")).createOrReplaceTempView("purchases_with_month")

## 6 SQL Queries

### Análisis de Ventas y Clientes con sqlContext

In [21]:
productos_mas_vendidos = sqlContext.sql("""
    SELECT Description, SUM(Quantity) AS TotalQuantity
    FROM purchases
    GROUP BY Description
    ORDER BY TotalQuantity DESC
    LIMIT 10
""")
productos_mas_vendidos.write.mode("overwrite").csv("productos_mas_vendidos.csv", header=True)


In [22]:
productos_mas_vendidos.show()

+--------------------+-------------+
|         Description|TotalQuantity|
+--------------------+-------------+
|PAPER CRAFT , LIT...|        80995|
|MEDIUM CERAMIC TO...|        78033|
|WORLD WAR 2 GLIDE...|        54951|
|JUMBO BAG RED RET...|        48371|
|WHITE HANGING HEA...|        37872|
|      POPCORN HOLDER|        36749|
|PACK OF 72 RETROS...|        36396|
|ASSORTED COLOUR B...|        36362|
|  RABBIT NIGHT LIGHT|        30739|
|MINI PAINT SET VI...|        26633|
+--------------------+-------------+



1. Productos con Mayor Cantidad Vendida
PAPER CRAFT, LITHOGRAPHIC: Este producto es el más vendido con 80,995 unidades, lo que indica una alta demanda o popularidad entre los clientes.
MEDIUM CERAMIC TOADSTOOL: El segundo producto más vendido, con 78,033 unidades, sugiere que los artículos decorativos cerámicos son populares.
WORLD WAR 2 GLIDER PLANES: Con 54,951 unidades vendidas, este juguete parece ser muy popular.
JUMBO BAG RED RETROSPOT: 48,371 unidades vendidas sugieren una alta demanda para este tipo de bolsas grandes.
WHITE HANGING HEART: Un artículo decorativo, con 37,872 unidades vendidas.
2. Variedad de Productos
La lista incluye una variedad de productos, desde artículos decorativos (como cerámica y artículos colgantes) hasta juguetes (como los aviones planeadores de la Segunda Guerra Mundial) y productos funcionales (como las bolsas Jumbo y los soportes para palomitas de maíz).

3. Análisis de Demanda
La alta cantidad vendida de estos productos puede ser indicativa de tendencias de consumo y preferencias de los clientes. Los artículos decorativos y los juguetes parecen ser especialmente populares.

4. Implicaciones para el Inventario
Los productos en esta lista deberían ser considerados para reabastecimiento frecuente para evitar agotamiento de existencias. La tienda puede beneficiarse de mantener un stock adecuado de estos artículos populares.

5. Estrategias de Marketing
Los resultados sugieren que estos productos podrían ser destacados en campañas de marketing y promociones para aumentar aún más las ventas. Además, se pueden ofrecer descuentos o paquetes promocionales con estos artículos para atraer más clientes.

In [24]:
# Productos más devueltos
productos_mas_devueltos = sqlContext.sql("""
    SELECT Description, SUM(Quantity) AS TotalQuantity
    FROM returns
    GROUP BY Description
    ORDER BY TotalQuantity ASC
    LIMIT 10
""")
productos_mas_devueltos.write.mode("overwrite").csv("productos_mas_devueltos.csv", header=True)

In [25]:
productos_mas_devueltos.show()

+--------------------+-------------+
|         Description|TotalQuantity|
+--------------------+-------------+
|PAPER CRAFT , LIT...|       -80995|
|MEDIUM CERAMIC TO...|       -74494|
|ROTATING SILVER A...|        -9376|
|              Manual|        -4066|
|FAIRY CAKE FLANNE...|        -3150|
|WHITE HANGING HEA...|        -2578|
|GIN + TONIC DIET ...|        -2030|
|   HERB MARKER BASIL|        -1527|
|FELTCRAFT DOLL MOLLY|        -1447|
|TEA TIME PARTY BU...|        -1424|
+--------------------+-------------+



1. Productos con Mayor Cantidad Devuelta
PAPER CRAFT, LITHOGRAPHIC: Este producto tiene la mayor cantidad de devoluciones con -80,995 unidades. Esto es significativo, ya que también es el producto más vendido, lo que puede indicar problemas con su calidad o que es un artículo con alta rotación.
MEDIUM CERAMIC TOADSTOOL: El segundo producto más devuelto con -74,494 unidades. Similar al producto anterior, esto sugiere posibles problemas de calidad o insatisfacción del cliente.
ROTATING SILVER ANGEL: Con -9,376 unidades devueltas, este producto también muestra una cantidad considerable de devoluciones.
Manual: La devolución de -4,066 unidades de un manual sugiere que puede haber errores en el contenido o falta de utilidad para los clientes.
FAIRY CAKE FLANNEL ASSORTED: Con -3,150 unidades devueltas, este artículo podría tener problemas de calidad o presentación.
2. Comparación con Productos Más Vendidos
Es interesante notar que PAPER CRAFT, LITHOGRAPHIC y MEDIUM CERAMIC TOADSTOOL aparecen tanto en la lista de productos más vendidos como en la de productos más devueltos. Esto sugiere que aunque son populares, pueden tener problemas de calidad o no cumplir con las expectativas de los clientes.

3. Causas de las Devoluciones
Problemas de Calidad: La alta cantidad de devoluciones podría indicar problemas de calidad con estos productos. Los artículos como PAPER CRAFT, LITHOGRAPHIC y MEDIUM CERAMIC TOADSTOOL necesitan una revisión para identificar y corregir problemas de fabricación o material.
Expectativas del Cliente: Productos como ROTATING SILVER ANGEL y FAIRY CAKE FLANNEL ASSORTED pueden no estar cumpliendo con las expectativas de los clientes, lo que lleva a devoluciones. Es crucial revisar las descripciones de productos y la retroalimentación de los clientes para hacer los ajustes necesarios.
4. Estrategias para Reducir Devoluciones
Control de Calidad: Implementar controles de calidad más estrictos para los productos que tienen altas tasas de devolución.
Mejora de Descripciones: Asegurarse de que las descripciones de los productos sean precisas y reflejen fielmente lo que el cliente recibirá.
Encuestas de Satisfacción: Realizar encuestas de satisfacción post-compra para identificar problemas y mejorar los productos.
5. Impacto en el Negocio
Las devoluciones tienen un costo significativo para el negocio, tanto en términos de logística como de insatisfacción del cliente. Reducir las tasas de devolución mediante mejoras en calidad y comunicación puede resultar en ahorros significativos y en una mayor lealtad del cliente.

In [26]:
# Clientes más valiosos (compras)
clientes_mas_valiosos = sqlContext.sql("""
    SELECT CustomerID, SUM(Quantity * UnitPrice) AS TotalSpent
    FROM purchases
    WHERE CustomerID != -1
    GROUP BY CustomerID
    ORDER BY TotalSpent DESC
    LIMIT 10
""")
clientes_mas_valiosos.write.mode("overwrite").csv("clientes_mas_valiosos.csv", header=True)

In [27]:
clientes_mas_valiosos.show()

+----------+------------------+
|CustomerID|        TotalSpent|
+----------+------------------+
|     14646| 280206.0200000002|
|     18102|259657.29999999993|
|     17450| 194390.7900000002|
|     16446|          168472.5|
|     14911| 143711.1699999997|
|     12415|124914.52999999994|
|     14156|117210.07999999983|
|     17511| 91062.37999999992|
|     16029|  80850.8400000001|
|     12346|           77183.6|
+----------+------------------+



1. Clientes con Mayor Gasto Total
CustomerID 14646: Este cliente es el más valioso con un gasto total de 280,206.02. Su alto nivel de gasto sugiere que es un cliente muy activo y probablemente frecuente.
CustomerID 18102: Con un gasto total de 259,657.30, este cliente también muestra un alto nivel de compromiso con la tienda.
CustomerID 17450: Este cliente ha gastado 194,390.79, colocándolo como el tercer cliente más valioso.
2. Distribución del Gasto
El análisis muestra que los clientes más valiosos han gastado cantidades significativas, con los dos principales clientes gastando más de 250,000 cada uno. Los siguientes clientes también han gastado considerablemente, pero hay una caída notable después de los primeros tres clientes.

3. Estrategias de Retención y Lealtad
Dado que estos clientes representan un alto valor para el negocio, es crucial implementar estrategias específicas para retenerlos y aumentar su lealtad:

Programas de Lealtad: Ofrecer programas de recompensas para estos clientes, como descuentos exclusivos, acceso anticipado a nuevos productos, y ofertas personalizadas.
Atención Personalizada: Proveer un nivel de atención más personalizado, como asistencia dedicada, seguimiento post-compra, y encuestas de satisfacción específicas.
Promociones Exclusivas: Crear promociones exclusivas para estos clientes, incentivándolos a seguir comprando y aumentando el valor de sus compras.
4. Segmentación de Clientes
Perfil Demográfico y Comportamental: Analizar más a fondo el perfil de estos clientes para identificar patrones demográficos y comportamentales comunes que pueden ayudar a segmentar y dirigir mejor las campañas de marketing.
Productos Preferidos: Identificar qué productos compran más estos clientes para asegurar que el inventario esté optimizado y se realicen campañas de marketing dirigidas.
5. Impacto en el Negocio
Mantener y aumentar el gasto de estos clientes más valiosos puede tener un impacto significativo en los ingresos del negocio. Invertir en estrategias de retención y lealtad específicas para estos clientes puede generar un retorno sustancial de la inversión.

In [28]:
# Análisis temporal de ventas (compras)
ventas_temporales = sqlContext.sql("""
    SELECT Month, SUM(Quantity * UnitPrice) AS TotalSales
    FROM purchases_with_month
    GROUP BY Month
    ORDER BY Month
""")

ventas_temporales.write.mode("overwrite").csv("ventas_temporales.csv", header=True)

In [29]:
ventas_temporales.show()

+-------+------------------+
|  Month|        TotalSales|
+-------+------------------+
|2010-12| 746723.6099999641|
|2011-01| 558448.5600000155|
|2011-02|497026.41000002634|
|2011-03| 682013.9800000135|
|2011-04|492367.84100002423|
|2011-05| 722094.1000000141|
|2011-06| 689977.2300000192|
|2011-07|  680156.991000024|
|2011-08| 703510.5800000189|
|2011-09|1017592.7820000201|
|2011-10| 1069368.230000005|
|2011-11| 1456145.799999814|
|2011-12| 432701.0600000215|
+-------+------------------+



1. Picos de Ventas
Noviembre 2011 (2011-11): Con un total de ventas de 1,503,866.78, noviembre es el mes con las ventas más altas del año. Esto probablemente se deba a la temporada de compras de fin de año, que incluye eventos como el Black Friday y las compras navideñas.
Octubre 2011 (2011-10): Octubre también muestra un alto nivel de ventas con 1,151,263.73, lo que podría indicar preparativos anticipados para la temporada de fin de año.
Septiembre 2011 (2011-09): Con ventas de 1,056,431.29, septiembre presenta un aumento significativo, posiblemente debido a las compras de regreso a clases o promociones de fin de verano.
2. Temporada Baja
Febrero 2011 (2011-02): Con 522,545.56, febrero es el mes con las ventas más bajas. Esto es común después del periodo de fiestas, ya que los consumidores tienden a reducir sus gastos.
Abril 2011 (2011-04): Otro mes con ventas relativamente bajas es abril, con un total de 536,968.49. Este patrón puede ser típico después del gasto adicional en marzo (posiblemente por las vacaciones de primavera).
3. Tendencias Generales
Incremento Estacional: Las ventas muestran un incremento significativo hacia el final del año, especialmente en los últimos tres meses (septiembre, octubre y noviembre).
Estabilidad Relativa: Los meses de verano (junio, julio, agosto) presentan ventas relativamente estables, con ligeras variaciones, pero manteniendo un nivel alto.

Implicaciones para el Negocio
Optimización del Inventario:

Mayor Stock en Temporada Alta: Dado el aumento en las ventas en los últimos meses del año, es crucial preparar el inventario con suficiente antelación para satisfacer la demanda durante estos meses.
Control de Stock en Temporada Baja: Reducir los niveles de inventario en meses de ventas bajas como febrero y abril puede ayudar a reducir costos de almacenamiento.
Estrategias de Marketing:

Promociones de Fin de Año: Implementar campañas de marketing agresivas durante los meses de septiembre a noviembre para maximizar las ventas en la temporada alta.
Promociones en Temporada Baja: Considerar promociones especiales o descuentos en meses de ventas bajas para estimular la demanda.
Planificación Financiera:

Preparación para Fluctuaciones: Las empresas deben planificar financieramente para los picos y valles en las ventas, asegurando liquidez suficiente durante los meses de menor venta y maximización de beneficios durante los meses de alta venta.
Análisis Continuo:

Monitoreo de Tendencias: Continuar monitoreando las tendencias de ventas mensuales para ajustar estrategias en tiempo real y mejorar la previsión de demanda.